<a href="https://colab.research.google.com/github/aruaru0/kaggle_h_and_m/blob/main/H%26M%E3%82%AB%E3%82%B9%E3%82%BF%E3%83%9EID%E3%81%A7item2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from collections import defaultdict
import json
from gensim.models import word2vec
import multiprocessing
cpu = multiprocessing.cpu_count()

In [ ]:
transactions = pd.read_csv("/content/drive/MyDrive/datas/h_and_m/transactions_train.csv",dtype=str, encoding='utf8')
articles = pd.read_csv("/content/drive/MyDrive/datas/h_and_m/articles.csv",dtype=str, encoding='utf8')
customers = pd.read_csv("/content/drive/MyDrive/datas/h_and_m/customers.csv",dtype=str, encoding='utf8')

In [ ]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050830508474576264,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.03049152542372881,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.01523728813559322,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932203389830508,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932203389830508,2


In [ ]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,0108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,09,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,0108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,0108775051,0108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,0110065001,0110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,09,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,0110065002,0110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [ ]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


### transactionを商品を購入したカスタマーで分類

In [ ]:
artdict = defaultdict(list)

tot = len(transactions)
for i in tqdm.tqdm(range(len(transactions))) :
  _, customer_id, article_id, _, _ = transactions.iloc[i]
  artdict[article_id].append(customer_id)
  # if i%100 == 99 : break


with open('/content/drive/MyDrive/datas/h_and_m/custvec/artdict.json', 'w') as fp:
    json.dump(artdict, fp)

100%|██████████| 31788324/31788324 [47:15<00:00, 11210.43it/s]


### customersを年齢別で分類

In [ ]:
agelist = defaultdict(list)

tot = len(customers)
for i in tqdm.tqdm(range(len(customers))) :
  customer_id, _, _, _, _, age, _ = customers.iloc[i]
  agelist[age].append(customer_id)
  # if i%100 == 99 : break


with open('/content/drive/MyDrive/datas/h_and_m/custvec/age.json', 'w') as fp:
  json.dump(agelist, fp)

100%|██████████| 1371980/1371980 [02:11<00:00, 10421.98it/s]


### テキスト形式で保存

In [ ]:
# テキスト形式にしｔ保存
with open('articles.txt', 'w') as f:
    for e in tqdm.tqdm(artdict.values()) :
      l = " ".join(e)
      f.write(l + '\n')

with open('age.txt', 'w') as f:
    for e in tqdm.tqdm(agelist.values()) :
      l = " ".join(e)
      f.write(l + '\n')

!cp *.txt /content/drive/MyDrive/datas/h_and_m/custvec/

100%|██████████| 85/85 [00:00<00:00, 176.95it/s]


In [ ]:
!cat *.txt  > words.txt

### Word2vec

In [ ]:
sentences = word2vec.LineSentence('words.txt')

# word2vecのモデルを構築
#     sg: 1ならskip-gram(cbowではなく)
#     size: 単語ベクトルの次元数
#     window: ウィンドウサイズ(今回は1行に含まれる全ての単語を対象とするため大きな値を設定)
#     hs: 0ならネガティブサンプリング(1なら階層的ソフトマックス)
#     negative: ネガティブサンプリングする単語数(結果への影響が予測できなかったのでデフォルトの5に設定)
#     seed: 乱数シード(結果を安定させたいので固定値を設定)
#model = word2vec.Word2Vec(sentences, sg=1, size=100, window=100000, hs=0, negative=5, seed=42)
model = word2vec.Word2Vec(sentences,  size=128, window=100000, seed=42, min_count=1, workers=cpu)

In [ ]:
model.save("/content/drive/MyDrive/datas/h_and_m/custvec/word2vec.model")

### transacionsからカスタマーIDで、[商品コード, 日付]のリストを作成する

In [ ]:
custdict = defaultdict(list)

tot = len(transactions)
for i in tqdm.tqdm(range(len(transactions))) :
  date, customer_id, article_id, _, _ = transactions.iloc[i]
  custdict[customer_id].append([date, article_id])
  #if i%100 == 99 : break


with open('/content/drive/MyDrive/datas/h_and_m/custvec/custdict.json', 'w') as fp:
    json.dump(custdict, fp)

IndentationError: ignored

In [ ]:
 model = word2vec.Word2Vec.load("/content/drive/MyDrive/datas/h_and_m/custvec/word2vec.model")

In [ ]:
d = dict()
for i in tqdm.tqdm(range(len(customers))) :
  id = customers.iloc[i]['customer_id']
  ret = model.wv.most_similar(id, topn=5)
  d[id] = ret
  

 20%|██        | 276773/1371980 [7:03:14<27:54:48, 10.90it/s]


KeyboardInterrupt: ignored

In [ ]:
with open('/content/drive/MyDrive/datas/h_and_m/custvec/result.json', "w") as f:
  json.dump(d, f)